# Workflow Manager

The Workflow Manager Notebook provides a centralized location for preparing and running calculations according to the high-throughput workflow used by the NIST Interatomic Potentials Repository.

## 1. Settings

This section allows for specific run settings to be managed and selected.

In [1]:
from copy import deepcopy
from multiprocessing import Pool
from pathlib import Path

import numpy as np

import iprPy

from iprPy.workflow import prepare, multi_runners, process, fix_lammps_versions

from iprPy.tools import aslist

### 1.1 Machine-based settings

This is where the LAMMPS and MPI commands to use on different machines can be defined.

Note: some potential versions do not work with the most current LAMMPS due to changes either in the parameter file structure or how LAMMPS reads the files. There *should* be new versions of these potential parameter files in the repository, with the old versions marked as either "retracted" or "superseded". Should you wish to test the older versions, you will need to install and specify older LAMMPS commands and specify potential_status to be "all" when preparing the calculations.

In [2]:
allcommands = {}

My desktop settings

In [3]:
commands = allcommands['desktop'] = {}

# mpi_command should list all settings except use {np_per_runner} for the variable number of processors per runner
commands['mpi_command'] = 'mpiexec -localonly {np_per_runner}' 

# lammps_command should be most current LAMMPS on your computer - at least after 30 Oct 2019
commands['lammps_command'] = 'E:/LAMMPS/2020-03-03/bin/lmp_mpi'

########## Optional old LAMMPS versions #############

# SNAP version 1 needs LAMMPS between 8 Oct 2014 and 30 May 2017
commands['lammps_command_snap_1'] = 'E:/LAMMPS/2017-01-27/bin/lmp_mpi'

# SNAP version 2 needs LAMMPS between 3 Dec 2018 and 12 June 2019
commands['lammps_command_snap_2'] = 'E:/LAMMPS/2019-06-05/bin/lmp_mpi'

# Old needs LAMMPS before 30 Oct 2019
commands['lammps_command_old'] = 'E:/LAMMPS/2019-06-05/bin/lmp_mpi'

My laptop settings

In [4]:
commands = allcommands['laptop'] = {}

# mpi_command should list all settings except use {np_per_runner} for the variable number of processors per runner
commands['mpi_command'] = 'C:/Program Files/MPICH2/bin/mpiexec -localonly {np_per_runner}' 

# lammps_command should be most current LAMMPS on your computer - at least after 30 Oct 2019
commands['lammps_command'] = 'lmp_mpi'

########## Optional old LAMMPS versions #############

# SNAP version 1 needs LAMMPS between 8 Oct 2014 and 30 May 2017
commands['lammps_command_snap_1'] = 'C:/Program Files/LAMMPS/2017-01-27/bin/lmp_mpi'

# SNAP version 2 needs LAMMPS between 3 Dec 2018 and 12 June 2019
commands['lammps_command_snap_2'] = 'C:/Program Files/LAMMPS/2019-06-05/bin/lmp_mpi'

# Old needs LAMMPS before 30 Oct 2019
commands['lammps_command_old'] = 'C:/Program Files/LAMMPS/2019-06-05/bin/lmp_mpi'

Ruth cluster settings

In [5]:
commands = allcommands['ruth'] = {}

# mpi_command should list all settings except use {np_per_runner} for the variable number of processors per runner
commands['mpi_command'] = '/cluster/deb9/bin/mpirun -n {np_per_runner}' 

# lammps_command should be most current LAMMPS on your computer - at least after 30 Oct 2019
commands['lammps_command'] = 'lmp_mpi'

########## Optional old LAMMPS versions #############

# SNAP version 1 needs LAMMPS between 8 Oct 2014 and 30 May 2017
#commands['lammps_command_snap_1'] =

# SNAP version 2 needs LAMMPS between 3 Dec 2018 and 12 June 2019
#commands['lammps_command_snap_2'] = 

# Old needs LAMMPS before 30 Oct 2019
#commands['lammps_command_old'] = 

### 1.2 Default database settings

In case you want to use the same settings for multiple calculations

- __database_name__ is the name of the database to load and use, i.e. where the calculation records are stored.
- __run_directory_name__ is the name of the run directory to load and use, i.e. where the calculations are prepared to, and executed from if num_runners is > 0. 
- __num_runners__ is the number of runners to start up after preparing a calculation, i.e. how many calculations to run at the same time.  Setting this to 0 will only prepare calculations allowing runners to be started up externally.
- __np_per_runner__ is the number of cores/processors each runner is allowed, i.e. number of cores to use for an mpi call for each calculation.
- __run_location__ specifies the machine on which the calculations will be performed on. If num_runners = 0, calculations can be prepared on one system and copied over to another for running. 


In [2]:
default_database_name = 'master'
default_run_directory_name = 'master_4'
default_num_runners = 6
default_np_per_runner = 1
default_run_location = 'desktop'

- --

## 2. Structure-free calculations

These are the calculations that evaluate energies computed from the potentials that are not associated with crystal structures.

In [3]:
# This is a list of bad implementations that should ALWAYS issue errors
bad_imps = [
    # Invalid parameter files
    '1990--Ackland-G-J--Cu--LAMMPS--ipr1',
    '2009--Zhakhovskii-V-V--Al--LAMMPS--ipr1',
    '2009--Zhakhovskii-V-V--Au--LAMMPS--ipr1',
    '2015--Broqvist-P--Ce-O--LAMMPS--ipr1',
    
    # Incorrect MEAM parameters in JSON
    '2009--Kim-H-K--Fe-Ti-C--LAMMPS--ipr1',
    '2012--Jelinek-B--Al-Si-Mg-Cu-Fe--LAMMPS--ipr1',
    '2013--Gao-H--AgTaO3--LAMMPS--ipr1',
    '2014--Liyanage-L-S-I--Fe-C--LAMMPS--ipr1',
    '2015--Ko-W-S--Ni-Ti--LAMMPS--ipr1',
    '2015--Pascuet-M-I--Al-U--LAMMPS--ipr1',
]

### 2.1 Diatom scan

Database settings

In [25]:
# Specify database settings to use for the calculation
database_name = default_database_name
run_directory_name = default_run_directory_name
num_runners = 1
np_per_runner = default_np_per_runner
run_location = default_run_location

# Generic settings
global_kwargs = {}

# Set commands for this machine
commands = allcommands[run_location]
global_kwargs['lammps_command'] = commands['lammps_command']
if np_per_runner > 1:
    global_kwargs['mpi_command'] = commands['mpi_command'].format(np_per_runner=np_per_runner)

Calculation settings

In [26]:
# Set other generic settings
#global_kwargs['prototype_id'] = 'A1--Cu--fcc'
global_kwargs['maximum_r'] = '10.0',
global_kwargs['number_of_steps_r'] = '500'

# Potential-based modifiers
pot_kwargs = {}
pot_kwargs['status'] = 'all'
pot_kwargs['id'] = ['1998--Meyer-R--Fe--LAMMPS--ipr-2']
#pot_kwargs['pair_style'] = ['eam', 'eam/alloy', 'eam/fs', 'eam/cd']

Prepare and run

In [27]:
# Initialize pool of workers
if num_runners > 1:
    pool = Pool(num_runners)
else:
    pool = None
    
# Prepare
kwargs = deepcopy(global_kwargs)
for key in pot_kwargs:
    kwargs[f'potential_{key}'] = pot_kwargs[key]
      
prepare.diatom_scan.main(database_name, run_directory_name, **kwargs)

# Fix LAMMPS versions if needed
fix_lammps_versions(run_directory_name, commands)

# Run
multi_runners(database_name, run_directory_name, num_runners, pool=pool)

# Close pool
if pool is not None:
    pool.close()

Preparing diatom_scan
900 existing calculation records found
1 record combinations to check
1 new records to prepare

Starting 1 runners in master_4 for master
Runner started with pid 13744
No simulations left to run



In [28]:
database = iprPy.load_database(database_name)
records_df = database.get_records_df(style='calculation_diatom_scan')
records_df = records_df[~records_df.potential_LAMMPS_id.isin(bad_imps)]

In [29]:
np.unique(records_df[records_df.status=='error'].potential_LAMMPS_id)

array([], dtype=object)

### 2.2 Isolated atom

Database settings

In [8]:
# Specify database settings to use for the calculation
database_name = default_database_name
run_directory_name = default_run_directory_name
num_runners = default_num_runners
np_per_runner = default_np_per_runner
run_location = default_run_location

# Generic settings
global_kwargs = {}

# Set commands for this machine
global_kwargs['lammps_command'] = allcommands[run_location]['lammps_command']
if np_per_runner > 1:
    global_kwargs['mpi_command'] = allcommands[run_location]['mpi_command'].format(np_per_runner=np_per_runner)

Calculation settings

In [9]:
# Set other generic settings


# Potential-based modifiers
pot_kwargs = {}
pot_kwargs['status'] = 'all'
pot_kwargs['id'] = ['1998--Meyer-R--Fe--LAMMPS--ipr-2']
#pot_kwargs['pair_style'] = ['eam', 'eam/alloy', 'eam/fs', 'eam/cd']

Prepare and run

In [22]:
# Initialize pool of workers
if num_runners > 1:
    pool = Pool(num_runners)
else:
    pool = None
    
# Prepare
kwargs = deepcopy(global_kwargs)
for key in pot_kwargs:
    kwargs[f'intpot_{key}'] = pot_kwargs[key]

prepare.isolated_atom.main(database_name, run_directory_name, **kwargs)

# Fix LAMMPS versions if needed
fix_lammps_versions(run_directory_name, commands)

# Run
multi_runners(database_name, run_directory_name, num_runners, pool=pool)

# Close pool
if pool is not None:
    pool.close()

Preparing isolated_atom
333 existing calculation records found
1 matching interatomic potentials found
1 record combinations to check
1 new records to prepare

Starting 6 runners in master_4 for master



In [23]:
database = iprPy.load_database(database_name)
records_df = database.get_records_df(style='calculation_isolated_atom')
records_df = records_df[~records_df.potential_LAMMPS_id.isin(bad_imps)]

In [24]:
np.unique(records_df[records_df.status=='error'].potential_LAMMPS_id)

array([], dtype=object)

In [21]:
records_df[records_df.status=='error']

,LAMMPS_version,atomman_version,branch,energy,error,iprPy_version,key,potential_LAMMPS_id,potential_LAMMPS_key,potential_id,potential_key,script,status
333,3 Mar 2020,1.3.5,main,NaN,"Traceback (most recent call last):\r\n File ""...",0.10.2,694a24f9-aae6-43d4-9110-5e8c41968da8,1998--Meyer-R--Fe--LAMMPS--ipr-2,44339012-ff3b-46ef-bdfd-d10d8d473276,1998--Meyer-R-Entel-P--Fe,bfd028cc-3a49-4a21-9731-56cee899c095,calc_isolated_atom,error


- - -

## 3. Structure relaxation calculations

These are the calculations used to identify the relaxed structures on a per-potential basis. This set of calculations comprises a mini-workflow all to itself:

- E_vs_r_scan runs first
- relax_box, relax_dynamic are then prepared from E_vs_r_scan results and DFT structures
- relax_static are then prepared from E_vs_r_scan, relax_dynamic results and DFT structures
- crystal_space_group are then prepared from relax_box, relax_static results, DFT structures and prototypes

### 3.1 E_vs_r_scan

Database settings

In [18]:
# Specify database settings to use for the calculation
database_name = default_database_name
run_directory_name = default_run_directory_name
num_runners = default_num_runners
np_per_runner = default_np_per_runner
run_location = default_run_location

# Generic settings
global_kwargs = {}

# Set commands for this machine
global_kwargs['lammps_command'] = allcommands[run_location]['lammps_command']
if np_per_runner > 1:
    global_kwargs['mpi_command'] = allcommands[run_location]['mpi_command'].format(np_per_runner=np_per_runner)

Calculation settings

In [19]:
# Set other generic settings
#global_kwargs['prototype_id'] = 'A1--Cu--fcc'

# Potential-based modifiers
pot_kwargs = {}
pot_kwargs['status'] = 'all'
pot_kwargs['id'] = ['2021--Wang-G--Au-Rh--LAMMPS--ipr1']
#pot_kwargs['pair_style'] = ['eam', 'eam/alloy', 'eam/fs', 'eam/cd']

Prepare and run

In [20]:
# Initialize pool of workers
if num_runners > 1:
    pool = Pool(num_runners)
else:
    pool = None

# Prepare
kwargs = deepcopy(global_kwargs)
for key in pot_kwargs:
    kwargs[f'prototype_potential_{key}'] = pot_kwargs[key]

# Prepare bop potentials
pair_style = kwargs.pop('prototype_potential_pair_style', None)
if pair_style is None or 'bop' in pair_style:
    try:
        prepare.E_vs_r_scan.bop(database_name, run_directory_name, **kwargs)
    except:
        print('No bops to give')

# Prepare other potentials
if pair_style is not None:
    kwargs['prototype_potential_pair_style'] = pair_style
prepare.E_vs_r_scan.main(database_name, run_directory_name, **kwargs)

# Fix LAMMPS versions if needed
fix_lammps_versions(run_directory_name, commands)

# Run
multi_runners(database_name, run_directory_name, num_runners, pool=pool)

# Close pool
if pool is not None:
    pool.close()

Preparing E_vs_r_scan
12306 existing calculation records found
No bops to give
Preparing E_vs_r_scan
12306 existing calculation records found
36 record combinations to check
36 new records to prepare

Starting 6 runners in master_4 for master



In [21]:
database = iprPy.load_database(database_name)
records_df = database.get_records_df(style='calculation_E_vs_r_scan')
records_df = records_df[~records_df.potential_LAMMPS_id.isin(bad_imps)]

In [22]:
np.unique(records_df[records_df.status=='error'].potential_LAMMPS_id)

array([], dtype=object)

### 3.2 relax box

Database settings

In [23]:
# Specify database settings to use for the calculation
database_name = default_database_name
run_directory_name = default_run_directory_name
num_runners = default_num_runners
np_per_runner = default_np_per_runner
run_location = default_run_location

# Generic settings
global_kwargs = {}

# Set commands for this machine
global_kwargs['lammps_command'] = allcommands[run_location]['lammps_command']
if np_per_runner > 1:
    global_kwargs['mpi_command'] = allcommands[run_location]['mpi_command'].format(np_per_runner=np_per_runner)

Calculation settings

In [24]:
# Set other generic settings
#global_kwargs['prototype_id'] = 'A1--Cu--fcc'

# Potential-based modifiers
pot_kwargs = {}
pot_kwargs['status'] = 'all'
pot_kwargs['id'] = ['2021--Wang-G--Au-Rh--LAMMPS--ipr1']
#pot_kwargs['pair_style'] = ['eam', 'eam/alloy', 'eam/fs', 'eam/cd']

Prepare and run

In [25]:
# Initialize pool of workers
if num_runners > 1:
    pool = Pool(num_runners)
else:
    pool = None

# Prepare
kwargs = deepcopy(global_kwargs)
for key in pot_kwargs:
    kwargs[f'reference_potential_{key}'] = pot_kwargs[key] 
    kwargs[f'parent_potential_{key}'] = pot_kwargs[key]

prepare.relax_box.main(database_name, run_directory_name, **kwargs)

# Fix LAMMPS versions if needed
fix_lammps_versions(run_directory_name, commands)

# Run
multi_runners(database_name, run_directory_name, num_runners, pool=pool)

# Close pool
if pool is not None:
    pool.close()

Preparing relax_box
43940 existing calculation records found
57 record combinations to check
57 new records to prepare

Starting 6 runners in master_4 for master



In [26]:
database = iprPy.load_database(database_name)
records_df = database.get_records_df(style='calculation_relax_box')
records_df = records_df[~records_df.potential_LAMMPS_id.isin(bad_imps)]

In [27]:
for error in np.unique(records_df[records_df.status=='error'].error):
    print('\n'.join(error.strip().split('\n')[-3:]))
    print()

  File "e:\python-packages\atomman\atomman\core\ElasticConstants.py", line 113, in Cij
    assert value.max() > 0.0, 'Cij values not valid'
AssertionError: Cij values not valid

  File "C:\Users\lmh1\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\linalg\linalg.py", line 97, in _raise_linalgerror_singular
    raise LinAlgError("Singular matrix")
numpy.linalg.LinAlgError: Singular matrix

  File "calc_relax_box.py", line 367, in calc_cij
    raise RuntimeError('Divergence of box dimensions to <= 0')
RuntimeError: Divergence of box dimensions to <= 0

  File "calc_relax_box.py", line 172, in relax_box
    raise RuntimeError('Divergence of box dimensions')
RuntimeError: Divergence of box dimensions

  File "calc_relax_box.py", line 174, in relax_box
    raise RuntimeError('Divergence of box dimensions')
RuntimeError: Divergence of box dimensions

  File "calc_relax_box.py", line 176, in relax_box
    raise RuntimeError('Divergence of box dimensions')
RuntimeError: Divergence of 

### 3.3 relax dynamic

Database settings

In [28]:
# Specify database settings to use for the calculation
database_name = default_database_name
run_directory_name = default_run_directory_name
num_runners = default_num_runners
np_per_runner = default_np_per_runner
run_location = default_run_location

# Generic settings
global_kwargs = {}

# Set commands for this machine
global_kwargs['lammps_command'] = allcommands[run_location]['lammps_command']
if np_per_runner > 1:
    global_kwargs['mpi_command'] = allcommands[run_location]['mpi_command'].format(np_per_runner=np_per_runner)

Calculation settings

In [29]:
# Set other generic settings
#global_kwargs['prototype_id'] = 'A1--Cu--fcc'

# Potential-based modifiers
pot_kwargs = {}
pot_kwargs['status'] = 'all'
pot_kwargs['id'] = ['2021--Wang-G--Au-Rh--LAMMPS--ipr1']
#pot_kwargs['pair_style'] = ['eam', 'eam/alloy', 'eam/fs', 'eam/cd']

Prepare and run

In [30]:
# Initialize pool of workers
if num_runners > 1:
    pool = Pool(num_runners)
else:
    pool = None

# Prepare
kwargs = deepcopy(global_kwargs)
for key in pot_kwargs:
    kwargs[f'reference_potential_{key}'] = pot_kwargs[key]  
    kwargs[f'parent_potential_{key}'] = pot_kwargs[key]

prepare.relax_dynamic.main(database_name, run_directory_name, **kwargs)

# Fix LAMMPS versions if needed
fix_lammps_versions(run_directory_name, commands)

# Run
multi_runners(database_name, run_directory_name, num_runners, pool=pool)

# Close pool
if pool is not None:
    pool.close()

Preparing relax_dynamic
44720 existing calculation records found
57 record combinations to check
57 new records to prepare

Starting 6 runners in master_4 for master



### 3.4 relax static

Database settings

In [31]:
# Specify database settings to use for the calculation
database_name = default_database_name
run_directory_name = default_run_directory_name
num_runners = default_num_runners
np_per_runner = default_np_per_runner
run_location = default_run_location

# Generic settings
global_kwargs = {}

# Set commands for this machine
global_kwargs['lammps_command'] = allcommands[run_location]['lammps_command']
if np_per_runner > 1:
    global_kwargs['mpi_command'] = allcommands[run_location]['mpi_command'].format(np_per_runner=np_per_runner)

Calculation settings

In [32]:
# Set other generic settings
#global_kwargs['prototype_id'] = 'A1--Cu--fcc'

# Potential-based modifiers
pot_kwargs = {}
pot_kwargs['status'] = 'all'
pot_kwargs['id'] = ['2021--Wang-G--Au-Rh--LAMMPS--ipr1']
#pot_kwargs['pair_style'] = ['eam', 'eam/alloy', 'eam/fs', 'eam/cd']

Prepare and run

In [33]:
# Initialize pool of workers
if num_runners > 1:
    pool = Pool(num_runners)
else:
    pool = None

# Prepare
kwargs = deepcopy(global_kwargs)
for key in pot_kwargs:
    kwargs[f'reference_potential_{key}'] = pot_kwargs[key]   
    kwargs[f'parent_potential_{key}'] = pot_kwargs[key]

prepare.relax_static.main(database_name, run_directory_name, **kwargs)

# Prepare relax_static from relax_dynamic
kwargs = deepcopy(global_kwargs)
for key in pot_kwargs:
    kwargs[f'archive_potential_{key}'] = pot_kwargs[key]

if 'parent_family' in kwargs:
    kwargs['archive_family'] = kwargs.pop('parent_family')

prepare.relax_static.from_dynamic(database_name, run_directory_name, **kwargs)

# Fix LAMMPS versions if needed
fix_lammps_versions(run_directory_name, commands)

# Run
multi_runners(database_name, run_directory_name, num_runners, pool=pool)

# Close pool
if pool is not None:
    pool.close()

Preparing relax_static
84714 existing calculation records found
57 record combinations to check
57 new records to prepare

Preparing relax_static
84771 existing calculation records found
57 record combinations to check
57 new records to prepare

Starting 6 runners in master_4 for master



### 3.5 crystal space group

Database settings

In [4]:
# Specify database settings to use for the calculation
database_name = default_database_name
run_directory_name = default_run_directory_name
num_runners = default_num_runners
np_per_runner = default_np_per_runner

# Generic settings
global_kwargs = {}

# Paths to compiled results
crystal_match_file = Path('csv', 'reference_prototype_match.csv')
all_crystals_file = Path('csv', f'{database_name}_all_crystals.csv')
unique_crystals_file = Path('csv', f'{database_name}_unique_crystals.csv')

Calculation settings

In [35]:
# Set other generic settings
#global_kwargs[]

Prepare and run

In [36]:
# Initialize pool of workers
if num_runners > 1:
    pool = Pool(num_runners)
else:
    pool = None

# Prepare
kwargs = deepcopy(global_kwargs)
prepare.crystal_space_group.relax(database_name, run_directory_name, **kwargs)

# Run
multi_runners(database_name, run_directory_name, num_runners, pool=pool)

# Close pool
if pool is not None:
    pool.close()

Preparing crystal_space_group
86832 existing calculation records found
81199 record combinations to check
137 new records to prepare

Starting 6 runners in master_4 for master



In [37]:
# Call process relaxation results and generate relaxed_crystal records
process.relaxed(database_name, crystal_match_file, all_crystals_file, unique_crystals_file)

43997 calculation_relax_box records found
 - 43997 are branch main
84828 calculation_relax_static records found
 - 43997 are branch main
 - 40831 are branch from_dynamic
82946 calculation_crystal_space_group records found
 - 19 are for prototypes
 - 5751 are for references
 - 77176 are for calculations
5751 references matched to prototypes
csv\master_all_crystals.csv updated
59820 untransformed crystals found in the compiled relaxation results
59718 relaxed_crystal records found in the database
 - 102 new results to add
59820 relaxed records found
 - 23952 currently have good standing
 - 23888 retain good standing
csv\master_unique_crystals.csv updated


- - -

## 4. Bulk crystal structure properties

These are non-defect properties that start from the unique relaxed crystals from section 3.

### 4.1 Elastic constants

Database settings

In [38]:
# Specify database settings to use for the calculation
database_name = default_database_name
run_directory_name = default_run_directory_name
num_runners = default_num_runners
np_per_runner = default_np_per_runner
run_location = default_run_location

# Generic settings
global_kwargs = {}

# Set commands for this machine
global_kwargs['lammps_command'] = allcommands[run_location]['lammps_command']
if np_per_runner > 1:
    global_kwargs['mpi_command'] = allcommands[run_location]['mpi_command'].format(np_per_runner=np_per_runner)

Calculation settings

In [39]:
#database = iprPy.load_database(database_name)
#pot_df = database.get_records_df(style='potential_LAMMPS')
#pot_ids = pot_df.id.tolist()
#len(pot_ids)

In [40]:
# Set other generic settings
#global_kwargs['parent_method'] = 'dynamic'
global_kwargs['parent_standing'] = 'good'

# Potential-based modifiers
pot_kwargs = {}
pot_kwargs['status'] = 'all'
#pot_kwargs['id'] = pot_ids[150:]
pot_kwargs['id'] = ['2021--Wang-G--Au-Rh--LAMMPS--ipr1']
#pot_kwargs['pair_style'] = ['eam', 'eam/alloy', 'eam/fs', 'eam/cd']

Prepare and run

In [41]:
# Initialize pool of workers
if num_runners > 1:
    pool = Pool(num_runners)
else:
    pool = None

# Prepare
kwargs = deepcopy(global_kwargs)
for key in pot_kwargs:
    kwargs[f'parent_potential_{key}'] = pot_kwargs[key]
prepare.elastic_constants_static.main(database_name, run_directory_name, **kwargs)

# Fix LAMMPS versions if needed
fix_lammps_versions(run_directory_name, commands)

# Run
multi_runners(database_name, run_directory_name, num_runners, pool=pool)

# Close pool
if pool is not None:
    pool.close()

Preparing elastic_constants_static
71550 existing calculation records found
114 record combinations to check
114 new records to prepare

Starting 6 runners in master_4 for master



## 5. Defect properties

These are crystalline defect structure properties based on the unique crystal structures identified in section #3.

### 5.1 Free surfaces

Database settings

In [7]:
# Specify database settings to use for the calculation
database_name = default_database_name
run_directory_name = 'master_3'
num_runners = 0
np_per_runner = default_np_per_runner
run_location = default_run_location

# Generic settings
global_kwargs = {}

# Set commands for this machine
global_kwargs['lammps_command'] = allcommands[run_location]['lammps_command']
if np_per_runner > 1:
    global_kwargs['mpi_command'] = allcommands[run_location]['mpi_command'].format(np_per_runner=np_per_runner)

Calculation settings

In [8]:
# Set other generic settings
global_kwargs['parent_method'] = 'dynamic'
global_kwargs['parent_standing'] = 'good'

# Prototype families to generate defects for
database = iprPy.load_database(database_name)
families = np.unique(database.get_records_df(style='free_surface').family)

# Potential-based modifiers
pot_kwargs = {}
pot_kwargs['status'] = 'all'
#pot_kwargs['id'] = pot_ids[15:]
#pot_kwargs['id'] = ['2021--Wang-G--Au-Rh--LAMMPS--ipr1']
#pot_kwargs['pair_style'] = ['eam', 'eam/alloy', 'eam/fs', 'eam/cd']

Prepare and run

In [9]:
# Initialize pool of workers
if num_runners > 1:
    pool = Pool(num_runners)
else:
    pool = None

# Prepare by each family (much faster)
for family in families:
    print(family)
    global_kwargs['parent_family'] = global_kwargs['defect_family'] = family
    kwargs = deepcopy(global_kwargs)
    for key in pot_kwargs:
        kwargs[f'parent_potential_{key}'] = pot_kwargs[key]
    try:
        prepare.surface_energy_static.main(database_name, run_directory_name, **kwargs)
    except:
        print('0 record combinations to check')
    
# Fix LAMMPS versions if needed
fix_lammps_versions(run_directory_name, commands)

# Run
multi_runners(database_name, run_directory_name, num_runners, pool=pool)

# Close pool
if pool is not None:
    pool.close()

A1--Cu--fcc
Preparing surface_energy_static
20505 existing calculation records found
332 matching interatomic potentials found
440 matching atomic parents found
13 matching defects
5720 record combinations to check
1755 new records to prepare

A15--beta-W
Preparing surface_energy_static
22261 existing calculation records found
332 matching interatomic potentials found
319 matching atomic parents found
16 matching defects
5104 record combinations to check
1264 new records to prepare

A2--W--bcc
Preparing surface_energy_static
23524 existing calculation records found
332 matching interatomic potentials found
269 matching atomic parents found
13 matching defects
3497 record combinations to check
1378 new records to prepare

A3'--alpha-La--double-hcp
Preparing surface_energy_static
24903 existing calculation records found
332 matching interatomic potentials found
297 matching atomic parents found
17 matching defects
5049 record combinations to check
799 new records to prepare

A3--Mg--hcp


### 5.2 Stacking Faults

Database settings

In [7]:
# Specify database settings to use for the calculation
database_name = default_database_name
run_directory_name = 'master_8'
num_runners = 1
np_per_runner = default_np_per_runner
run_location = default_run_location

# Generic settings
global_kwargs = {}

# Set commands for this machine
global_kwargs['lammps_command'] = allcommands[run_location]['lammps_command']
if np_per_runner > 1:
    global_kwargs['mpi_command'] = allcommands[run_location]['mpi_command'].format(np_per_runner=np_per_runner)

Calculation settings

In [8]:
# Set other generic settings
global_kwargs['parent_method'] = 'dynamic'
global_kwargs['parent_standing'] = 'good'

# Prototype families to generate defects for
database = iprPy.load_database(database_name)
families = np.unique(database.get_records_df(style='stacking_fault').family)

# Potential-based modifiers
pot_kwargs = {}
pot_kwargs['status'] = 'all'
#pot_kwargs['id'] = pot_ids[15:]
pot_kwargs['id'] = ['1999--Mishin-Y--Al--LAMMPS--ipr1']
#pot_kwargs['pair_style'] = ['eam', 'eam/alloy', 'eam/fs', 'eam/cd']

Prepare and run

In [9]:
# Initialize pool of workers
if num_runners > 1:
    pool = Pool(num_runners)
else:
    pool = None

# Prepare by each family (much faster)
for family in families:
    print(family)
    global_kwargs['parent_family'] = global_kwargs['defect_family'] = family
    kwargs = deepcopy(global_kwargs)
    for key in pot_kwargs:
        kwargs[f'parent_potential_{key}'] = pot_kwargs[key]
    prepare.stacking_fault_map_2D.main(database_name, run_directory_name, **kwargs)
    break

# Fix LAMMPS versions if needed
fix_lammps_versions(run_directory_name, commands)

# Run
multi_runners(database_name, run_directory_name, num_runners, pool=pool)

# Close pool
if pool is not None:
    pool.close()

A1--Cu--fcc
Preparing stacking_fault_map_2D
0 existing calculation records found
2 matching defects found
2 record combinations to check
2 new records to prepare

Starting 1 runners in master_8 for master
Runner started with pid 17264
No simulations left to run



### 5.3 Dislocation monopoles

Database settings

In [11]:
# Specify database settings to use for the calculation
database_name = default_database_name
run_directory_name = 'master_7'
num_runners = 0
np_per_runner = default_np_per_runner
run_location = default_run_location

# Generic settings
global_kwargs = {}

# Set commands for this machine
global_kwargs['lammps_command'] = allcommands[run_location]['lammps_command']
if np_per_runner > 1:
    global_kwargs['mpi_command'] = allcommands[run_location]['mpi_command'].format(np_per_runner=np_per_runner)

Calculation settings

In [12]:
# Set other generic settings
#global_kwargs['parent_strainrange'] = 1e-8

# Potential-based modifiers
pot_kwargs = {}
pot_kwargs['status'] = 'all'
#pot_kwargs['id'] = pot_ids[15:]
pot_kwargs['id'] = ['2006--Chamati-H--Fe--LAMMPS--ipr1']
#pot_kwargs['pair_style'] = ['eam', 'eam/alloy', 'eam/fs', 'eam/cd']

Prepare and run

In [13]:
# Initialize pool of workers
if num_runners > 1:
    pool = Pool(num_runners)
else:
    pool = None

# Prepare bcc screw    
kwargs = deepcopy(global_kwargs)
for key in pot_kwargs:
    kwargs[f'parent_potential_{key}'] = pot_kwargs[key]
prepare.dislocation_monopole.bcc_screw(database_name, run_directory_name, **kwargs)
prepare.dislocation_monopole.bcc_edge(database_name, run_directory_name, **kwargs)
prepare.dislocation_monopole.bcc_edge_112(database_name, run_directory_name, **kwargs)
prepare.dislocation_monopole.fcc_edge_100(database_name, run_directory_name, **kwargs)

# Fix LAMMPS versions if needed
fix_lammps_versions(run_directory_name, commands)

# Run
multi_runners(database_name, run_directory_name, num_runners, pool=pool)

# Close pool
if pool is not None:
    pool.close()

Preparing dislocation_monopole
2 existing calculation records found
1 matching interatomic potentials found
552 matching atomic archives found
1 matching defects
2 record combinations to check
0 new records to prepare

Preparing dislocation_monopole
2 existing calculation records found
1 matching interatomic potentials found
552 matching atomic archives found
1 matching defects
2 record combinations to check
2 new records to prepare

Preparing dislocation_monopole
4 existing calculation records found
1 matching interatomic potentials found
552 matching atomic archives found
1 matching defects
2 record combinations to check
2 new records to prepare

Preparing dislocation_monopole
6 existing calculation records found
1 matching interatomic potentials found
615 matching atomic archives found
1 matching defects
2 record combinations to check
2 new records to prepare



### 5.4 Dislocation arrays

Database settings

In [7]:
# Specify database settings to use for the calculation
database_name = default_database_name
run_directory_name = 'master_7'
num_runners = 0
np_per_runner = default_np_per_runner
run_location = default_run_location

# Generic settings
global_kwargs = {}

# Set commands for this machine
global_kwargs['lammps_command'] = allcommands[run_location]['lammps_command']
if np_per_runner > 1:
    global_kwargs['mpi_command'] = allcommands[run_location]['mpi_command'].format(np_per_runner=np_per_runner)

Calculation settings

In [8]:
# Set other generic settings
#global_kwargs['parent_strainrange'] = 1e-8

# Potential-based modifiers
pot_kwargs = {}
pot_kwargs['status'] = 'all'
#pot_kwargs['id'] = pot_ids[15:]
pot_kwargs['id'] = ['2006--Chamati-H--Fe--LAMMPS--ipr1']
#pot_kwargs['pair_style'] = ['eam', 'eam/alloy', 'eam/fs', 'eam/cd']

Prepare and run

In [10]:
# Initialize pool of workers
if num_runners > 1:
    pool = Pool(num_runners)
else:
    pool = None

# Prepare bcc screw    
kwargs = deepcopy(global_kwargs)
for key in pot_kwargs:
    kwargs[f'parent_potential_{key}'] = pot_kwargs[key]
prepare.dislocation_periodic_array.fcc_edge_mix(database_name, run_directory_name, **kwargs)
prepare.dislocation_periodic_array.fcc_screw(database_name, run_directory_name, **kwargs)

# Fix LAMMPS versions if needed
fix_lammps_versions(run_directory_name, commands)

# Run
multi_runners(database_name, run_directory_name, num_runners, pool=pool)

# Close pool
if pool is not None:
    pool.close()

Preparing dislocation_periodic_array
4 existing calculation records found
1 matching interatomic potentials found
615 matching atomic archives found
1 matching defects
2 record combinations to check
0 new records to prepare

Preparing dislocation_periodic_array
4 existing calculation records found
1 matching interatomic potentials found
615 matching atomic archives found
1 matching defects
2 record combinations to check
0 new records to prepare

